### Problem Definition

* What is an LNA?

* What is Noise Figure?

* Why 2.45 GHz?

In [1]:
# Importing necessary modules
import numpy as np
import matplotlib.pyplot as plt
import scipy

## Simulation Plan